<a href="https://colab.research.google.com/github/FourHan/tensorflow_practice/blob/master/seq2seq_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 刚开始会提示错误，但是再执行一下就不会出错了
!pip install tf-nightly


In [0]:
"水电费水电费连接"

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__   

'1.14.0'

In [0]:
!pip install tensorflow==1.14

In [0]:
!git clone https://github.com/tensorflow/nmt.git

Cloning into 'nmt'...
remote: Enumerating objects: 1283, done.
remote: Total 1283 (delta 0), reused 0 (delta 0), pack-reused 1283
Receiving objects: 100% (1283/1283), 1.24 MiB | 1.39 MiB/s, done.
Resolving deltas: 100% (918/918), done.


In [0]:
!ls

data1  nmt  nmt1  sample_data


In [0]:
!ls nmt

CONTRIBUTING.md  LICENSE  nmt  README.md


In [0]:
!ls nmt/nmt

attention_model.py  inference_test.py  model_test.py  standard_hparams
g3doc		    __init__.py        nmt.py	      testdata
gnmt_model.py	    model_helper.py    nmt_test.py    train.py
inference.py	    model.py	       scripts	      utils


In [0]:
!cat nmt/nmt/scripts/download_iwslt15.sh

#!/bin/sh
# Download small-scale IWSLT15 Vietnames to English translation data for NMT
# model training.
#
# Usage:
#   ./download_iwslt15.sh path-to-output-dir
#
# If output directory is not specified, "./iwslt15" will be used as the default
# output directory.
OUT_DIR="${1:-iwslt15}"
SITE_PREFIX="https://nlp.stanford.edu/projects/nmt/data"

mkdir -v -p $OUT_DIR

# Download iwslt15 small dataset from standford website.
echo "Download training dataset train.en and train.vi."
curl -o "$OUT_DIR/train.en" "$SITE_PREFIX/iwslt15.en-vi/train.en"
curl -o "$OUT_DIR/train.vi" "$SITE_PREFIX/iwslt15.en-vi/train.vi"

echo "Download dev dataset tst2012.en and tst2012.vi."
curl -o "$OUT_DIR/tst2012.en" "$SITE_PREFIX/iwslt15.en-vi/tst2012.en"
curl -o "$OUT_DIR/tst2012.vi" "$SITE_PREFIX/iwslt15.en-vi/tst2012.vi"

echo "Download test dataset tst2013.en and tst2013.vi."
curl -o "$OUT_DIR/tst2013.en" "$SITE_PREFIX/iwslt15.en-vi/tst2013.en"
curl -o "$OUT_DIR/tst2013.vi" "$SITE_PREFIX/iwslt15.en-vi/tst2013

In [0]:
!nmt/nmt/scripts/download_iwslt15.sh ./data

mkdir: created directory './data'
Download training dataset train.en and train.vi.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12.9M  100 12.9M    0     0  2308k      0  0:00:05  0:00:05 --:--:-- 3225k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.2M  100 17.2M    0     0  2903k      0  0:00:06  0:00:06 --:--:-- 4190k
Download dev dataset tst2012.en and tst2012.vi.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  136k  100  136k    0     0    99k      0  0:00:01  0:00:01 --:--:--   99k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


In [0]:
!ls

data  data1  nmt  nmt1	sample_data


In [0]:
!ls data

train.en  tst2012.en  tst2013.en  vocab.en
train.vi  tst2012.vi  tst2013.vi  vocab.vi


In [0]:
!head  -10 data/train.en

Rachel Pike : The science behind a climate headline
In 4 minutes , atmospheric chemist Rachel Pike provides a glimpse of the massive scientific effort behind the bold headlines on climate change , with her team -- one of thousands who contributed -- taking a risky flight over the rainforest in pursuit of data on a key molecule .
I &apos;d like to talk to you today about the scale of the scientific effort that goes into making the headlines you see in the paper .
Headlines that look like this when they have to do with climate change , and headlines that look like this when they have to do with air quality or smog .
They are both two branches of the same field of atmospheric science .
Recently the headlines looked like this when the Intergovernmental Panel on Climate Change , or IPCC , put out their report on the state of understanding of the atmospheric system .
That report was written by 620 scientists from 40 countries .
They wrote almost a thousand pages on the topic .
And all of tho

In [0]:
!head -10 data/train.vi

Khoa học đằng sau một tiêu đề về khí hậu
Trong 4 phút , chuyên gia hoá học khí quyển Rachel Pike giới thiệu sơ lược về những nỗ lực khoa học miệt mài đằng sau những tiêu đề táo bạo về biến đổi khí hậu , cùng với đoàn nghiên cứu của mình -- hàng ngàn người đã cống hiến cho dự án này -- một chuyến bay mạo hiểm qua rừng già để tìm kiếm thông tin về một phân tử then chốt .
Tôi muốn cho các bạn biết về sự to lớn của những nỗ lực khoa học đã góp phần làm nên các dòng tít bạn thường thấy trên báo .
Có những dòng trông như thế này khi bàn về biến đổi khí hậu , và như thế này khi nói về chất lượng không khí hay khói bụi .
Cả hai đều là một nhánh của cùng một lĩnh vực trong ngành khoa học khí quyển .
Các tiêu đề gần đây trông như thế này khi Ban Điều hành Biến đổi khí hậu Liên chính phủ , gọi tắt là IPCC đưa ra bài nghiên cứu của họ về hệ thống khí quyển .
Nghiên cứu được viết bởi 620 nhà khoa học từ 40 quốc gia khác nhau .
Họ viết gần 1000 trang về chủ đề này .
Và tất cả các trang đều được xem 

In [0]:
!head -10 data/vocab.vi

<unk>
<s>
</s>
Khoa
học
đằng
sau
một
tiêu
đề


In [0]:
!hed -10 data/vocab.en

/bin/bash: hed: command not found


In [0]:
!apt install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (36.9 kB/s)
Selecting previously unselected package tree.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
!tree .

.
├── data
│   ├── train.en
│   ├── train.vi
│   ├── tst2012.en
│   ├── tst2012.vi
│   ├── tst2013.en
│   ├── tst2013.vi
│   ├── vocab.en
│   └── vocab.vi
├── data1
│   ├── train.en
│   ├── train.vi
│   ├── tst2012.en
│   ├── tst2012.vi
│   ├── tst2013.en
│   ├── tst2013.vi
│   ├── vocab.en
│   └── vocab.vi
├── nmt
│   ├── CONTRIBUTING.md
│   ├── LICENSE
│   ├── nmt
│   │   ├── attention_model.py
│   │   ├── g3doc
│   │   │   └── img
│   │   │       ├── attention_equation_0.jpg
│   │   │       ├── attention_equation_1.jpg
│   │   │       ├── attention_mechanism.jpg
│   │   │       ├── attention_vis.jpg
│   │   │       ├── encdec.jpg
│   │   │       ├── greedy_dec.jpg
│   │   │       └── seq2seq.jpg
│   │   ├── gnmt_model.py
│   │   ├── inference.py
│   │   ├── inference_test.py
│   │   ├── __init__.py
│   │   ├── model_helper.py
│   │   ├── model.py
│   │   ├── model_test.py
│   │   ├── nmt.py
│   │   ├── nmt_test.py
│   │   ├── __pycache__
│   │   │   ├── attention_model.cpython-36.py

In [0]:
!mkdir /tmp/nmt_attention_model

mkdir: cannot create directory ‘/tmp/nmt_attention_model’: File exists


In [0]:
!ls /tmp/nmt_attention_model

best_bleu  hparams  log_1578640477  log_1578642392  train_log


In [0]:
!ls data


train.en  tst2012.en  tst2013.en  vocab.en
train.vi  tst2012.vi  tst2013.vi  vocab.vi


In [0]:
!python3 -m nmt.nmt.nmt \
    --attention=scaled_luong \
    --src=vi --tgt=en \
    --vocab_prefix=./data/vocab  \
    --train_prefix=./data/train \
    --dev_prefix=./data/tst2012  \
    --test_prefix=./data/tst2013 \
    --out_dir=./data/nmt_attention_model \
    --num_train_steps=12000 \
    --steps_per_stats=100 \
    --num_layers=2 \
    --num_units=128 \
    --dropout=0.2 \
    --metrics=bleu

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd

In [0]:
!python

In [0]:
%tensorflow_version 1.x
import tensorflow as tf

In [0]:
tf.__version__

'1.14.0'